## Simple linear regression

Watch the video below to learn about simple linear regression.

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/PaFPbb66DxQ?start=24" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe> 

[Explaining Linear Regression, by StatQuest](https://www.youtube.com/embed/PaFPbb66DxQ?start=24)
```


Then read the following sections to learn more about simple linear regression with examples in the ISLR book.

### Import libraries and load data

<!-- Firstly, we will import the required libraries and load the `Advertising` dataset. -->

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.formula.api import ols

%matplotlib inline
plt.style.use("seaborn-white")

**Load `Advertising` datasets**

Datasets available on https://www.statlearning.com/resources-first-edition

In [ ]:
data_url = "https://github.com/pykale/transparentML/raw/main/data/Advertising.csv"

advertising_df = pd.read_csv(
    data_url, header=0, index_col=0
)  # read Boston data as pandas data frame
advertising_df.info()

In [ ]:
advertising_df.head()

Simple linear regression assumes that there is a approximately a linear relationship between a quantitative response $Y$ and a single predictor $X$. Mathematically, the linear relationship can be expressed as

\begin{equation}
Y \approx \beta_0 + \beta_1 X
\end{equation}

where $\beta_0$ and $\beta_1$ are two unknown constants that represent the *intercept* and *slope* of the linear model. Together, $\beta_0$ and $\beta_1$ are called the *model coefficients*, or *parameters*. We can describe the linear relationship as *regressing* $Y$ onto $X$. For example, $X$ may represent `TV` advertising and $Y$ may represent `sales`. Then we can regress sales onto TV by fitting the model

<!-- \begin{equation} -->
$$
\textrm{sales} \approx \beta_0 + \beta_1 \times \textrm{TV}
$$
<!-- \end{equation} -->



### Estimating the coefficients

The goal of simple linear regression is to estimate the unknown parameters $\beta_0$ and $\beta_1$ from the data. Let 

$$
(x_1, y_1), (x_2, y_2), \ldots, (x_n, y_n)
$$

be the $n$ observations in the dataset, and $\hat{\beta}_0$ and $\hat{\beta}_1$ denote the estimated values of $\beta_0$ and $\beta_1$, respectively. The estimated values of $\hat{\beta}_0$ and $\hat{\beta}_1$ can be obtained by minimising the *residual sum of squares* (RSS)


\begin{equation}
\text{RSS} = \sum_{i=1}^n (y_i - \hat{y}_i)^2 = \sum_{i=1}^n (y_i - \hat{\beta}_0 - \hat{\beta}_1 x_i)^2,
\end{equation}


where $y_i$ is the $i\text{th}$ observation of the response $Y$, $\hat{y}_i$ is the $i\text{th}$ observation of the predicted response $\hat{Y}$, and $x_i$ is the $i\text{th}$ observation of the predictor $X$. The least squares estimates of $\beta_0$ and $\beta_1$ are given by


\begin{equation}
\hat{\beta}_1 = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2},
\end{equation}

\begin{equation}
\hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x}.
\end{equation}

where $\bar{x}$ and $\bar{y}$ are the sample means of $X$ and $Y$ respectively. The least squares estimates of $\beta_0$ and $\beta_1$ are obtained by minimising the RSS. The least squares line is given by


\begin{equation}
\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x
\end{equation}

The least squares line is the line that minimises the sum of squared residuals. The least squares line is also known as the *regression line*.

**Figure 3.1 - Least squares fit**

This figure shows the least squares fit of `sales` onto `TV` for the `Advertising` dataset. The objective is to minimising the sum of squared residuals, which is the sum of the vertical distances between the data points (red dots) and the least squares line (blue line). The least squares line is the line that minimises the sum of squared residuals. The least squares line is also known as the *regression line*.

In [ ]:
sns.regplot(
    x=advertising_df.TV,
    y=advertising_df.Sales,
    order=1,
    ci=None,
    scatter_kws={"color": "r", "s": 9},
)
plt.xlim(-10, 310)
plt.ylim(ymin=0)
plt.show()

**Figure 3.2 - Regression coefficients - RSS**

Firstly, fit a simple linear regression model using the scikit-learn library. The `.fit()` method takes two arguments, the first is the predictor variable and the second is the response variable. The `.fit()` method returns an object that contains the estimated coefficients. The `.intercept_` and `.coef_` attributes of the fitted model can be used to obtain the estimated intercept ($\beta_0$) and slope ($\beta_1$) of the regression line. 

Note that the text in the book describes the coefficients based on unnormalised data, whereas the plot shows the model based on normalised data. The latter is visually more appealing for explaining the concept of a minimum RSS. I think that, in order not to confuse the reader, the values on the axis of the `beta_0` coefficients have been changed to correspond with the text. The axes on the plots below are unaltered.

In [ ]:
# Regression coefficients (Ordinary Least Squares)
regr = LinearRegression()

X = scale(advertising_df.TV, with_mean=True, with_std=False).reshape(-1, 1)
y = advertising_df.Sales

regr.fit(X, y)
print(regr.intercept_)
print(regr.coef_)

In [ ]:
# Create grid coordinates for plotting
beta_0 = np.linspace(regr.intercept_ - 2, regr.intercept_ + 2, 50)
beta_1 = np.linspace(regr.coef_ - 0.02, regr.coef_ + 0.02, 50)
xx, yy = np.meshgrid(beta_0, beta_1, indexing="xy")
Z = np.zeros((beta_0.size, beta_1.size))

# Calculate Z-values (RSS) based on grid of coefficients
for (i, j), v in np.ndenumerate(Z):
    Z[i, j] = ((y - (xx[i, j] + X.ravel() * yy[i, j])) ** 2).sum() / 1000

# minimised RSS
min_RSS = r"$\beta_0$, $\beta_1$ for minimised RSS"
min_rss = (
    np.sum((regr.intercept_ + regr.coef_ * X - y.values.reshape(-1, 1)) ** 2) / 1000
)
min_rss

In [ ]:
fig = plt.figure(figsize=(15, 6))
fig.suptitle("RSS - Regression coefficients", fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, projection="3d")

# Left plot
CS = ax1.contour(xx, yy, Z, cmap=plt.cm.Set1, levels=[2.15, 2.2, 2.3, 2.5, 3])
ax1.scatter(regr.intercept_, regr.coef_[0], c="r", label=min_RSS)
ax1.clabel(CS, inline=True, fontsize=10, fmt="%1.1f")

# Right plot
ax2.plot_surface(xx, yy, Z, rstride=3, cstride=3, alpha=0.3)
ax2.contour(
    xx,
    yy,
    Z,
    zdir="z",
    offset=Z.min(),
    cmap=plt.cm.Set1,
    alpha=0.4,
    levels=[2.15, 2.2, 2.3, 2.5, 3],
)
ax2.scatter3D(regr.intercept_, regr.coef_[0], min_rss, c="r", label=min_RSS)
ax2.set_zlabel("RSS")
ax2.set_zlim(Z.min(), Z.max())
ax2.set_ylim(0.02, 0.07)

# settings common to both plots
for ax in fig.axes:
    ax.set_xlabel(r"$\beta_0$", fontsize=17)
    ax.set_ylabel(r"$\beta_1$", fontsize=17)
    ax.set_yticks([0.03, 0.04, 0.05, 0.06])
    ax.legend()

### Assessing the accuracy of the coefficient estimates

The linear relationship between $X$ and $Y$ can be written in an equation as

\begin{equation}
Y = \beta_0 + \beta_1 X + \epsilon,
\end{equation}

where $\epsilon$ is a random error term that represents the difference between the observed response $Y$ and the true response $\beta_0 + \beta_1 X$. The error term $\epsilon$ is assumed to be normally distributed with mean zero and constant variance $\sigma^2$. The coefficient estimates $\hat{\beta}_0$ and $\hat{\beta}_1$ are only estimates of the true coefficients $\beta_0$ and $\beta_1$. We can quantify the accuracy of the estimates by computing the *standard error* of the estimates. The standard error of $\hat{\beta}_1$ is given by 

\begin{equation}
\text{SE}(\hat{\beta}_{1})^2 = \frac{\sigma^2}{\sum_{i=1}^n (x_i - \bar{x})^2}, \quad \text{SE}(\hat{\beta}_0)^2 = \sigma^2 \left[\frac{1}{n} + \frac{\bar{x}^2}{\sum_{i=1}^n (x_i - \bar{x})^2} \right],
\end{equation}

where $\sigma^2$ is an estimate of the variance of the error term $\epsilon= Y - (\beta_0 + \beta_1 X) $, $\hat{y}_i$ is the $i\text{th}$ observation of the predicted response $\hat{Y}$, and $x_i$ is the $i\text{th}$ observation of the predictor $X$, $\bar{x}$ and $\bar{y}$ are the sample means of $X$ and $Y$ respectively. Please note the $\sigma^2$ is an estimation from the dataset of the whole population variance. This estimate is called the *residual standard error* (RSE)

\begin{equation}
\text{RSE} = \sqrt{\frac{1}{n-2}\text{RSS}} = \sqrt{\frac{1}{n-2} \sum_{i=1}^n (y_i - \hat{y}_i)^2}.
\end{equation}


The standard error of $\hat{\beta}_1$ is a measure of the variability of the estimate of $\beta_1$ due to sampling error. The standard error of $\hat{\beta}_0$ is a measure of the variability of the estimate of $\beta_0$ due to sampling error. The standard error of $\hat{\beta}_1$ is also known as the *standard error of the regression*.


**Confidence interval on page 67 & Table 3.1 & 3.2 - Statsmodels**

In [ ]:
est = ols("Sales ~ TV", advertising_df).fit()
est.summary().tables[1]

In [ ]:
# RSS with regression coefficients
(
    (advertising_df.Sales - (est.params[0] + est.params[1] * advertising_df.TV)) ** 2
).sum() / 1000

### Assessing the accuracy of the model

The accuracy of the linear model is dependent on the variability of the response $Y$ and the predictor $X$. The variability of $Y$ is measured by the variance of $Y$, denoted by $\sigma^2$. The variability of $X$ is measured by the variance of $X$, denoted by $\sigma_x^2$. The coefficient of determination, denoted by $R^2$, is defined as

\begin{equation}
R^2 = \frac{\text{TSS} - \text{RSS}}{\text{TSS}} = 1 - \frac{\text{RSS}}{\text{TSS}}
\end{equation}

where $\text{TSS} = \sum_{i=1}^n (y_i - \bar{y})^2$ *is the total sum of squares*. The coefficient of determination is a proportion that measures the proportion of the variability in the response that is explained by the linear model. The coefficient of determination is always between 0 and 1. The coefficient of determination is 0 when the regression line does not fit the data at all, and the coefficient of determination is 1 when the regression line perfectly fits the data. The coefficient of determination is also known as the *coefficient of multiple determination*. The coefficient of determination is a measure of the goodness of fit of the linear model. The coefficient of determination is also known as the *coefficient of multiple determination*. The coefficient of determination is a measure of the goodness of fit of the linear model.

Watch the video below to learn more about $R^2$

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/2AQKmw14mHM?start=16" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe> 

[Explaining $R^2$, by StatQuest](https://www.youtube.com/embed/2AQKmw14mHM?start=16)
```

**Table 3.1 & 3.2 - Scikit-learn**

In [ ]:
regr = LinearRegression()

X = advertising_df.TV.values.reshape(-1, 1)
y = advertising_df.Sales

regr.fit(X, y)
print(regr.intercept_)
print(regr.coef_)

In [ ]:
Sales_pred = regr.predict(X)
r2_score(y, Sales_pred)